In [139]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [140]:
df = pd.read_csv('data/cfb_data.csv')
df.describe().round(2)

,id,year,week,postseason,id_home_team,id_away_team,points_home,points_away,completed,conference_game,home_team_id,away_team_id,home_team_ranking,away_team_ranking
count,97109.0,97109.00,97109.00,97109.00,97109.00,97109.00,96744.00,96744.00,97109.00,97109.00,97109.00,97109.00,11363.00,8835.00
mean,48555.0,1972.60,6.83,0.02,49423.46,77702.19,24.22,16.91,1.00,0.53,49423.46,77702.19,11.44,11.83
std,28033.1,38.14,3.50,0.13,214397.67,265522.78,16.46,14.17,0.03,0.50,214397.67,265522.78,6.74,6.74
min,1.0,1869.00,1.00,0.00,2.00,2.00,0.00,0.00,0.00,0.00,2.00,2.00,1.00,1.00
25%,24278.0,1941.00,4.00,0.00,142.00,154.00,12.00,6.00,1.00,0.00,142.00,154.00,6.00,6.00
50%,48555.0,1978.00,7.00,0.00,252.00,295.00,22.00,14.00,1.00,1.00,252.00,295.00,11.00,12.00
75%,72832.0,2008.00,10.00,0.00,2348.00,2460.00,34.00,26.00,1.00,1.00,2348.00,2460.00,17.00,17.00
max,97109.0,2022.00,20.00,1.00,1000929.00,1000932.00,222.00,158.00,1.00,1.00,1000929.00,1000932.00,25.00,25.00


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97109 entries, 0 to 97108
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     97109 non-null  int64  
 1   year                   97109 non-null  int64  
 2   week                   97109 non-null  int64  
 3   postseason             97109 non-null  int64  
 4   id_home_team           97109 non-null  int64  
 5   id_away_team           97109 non-null  int64  
 6   points_home            96744 non-null  float64
 7   points_away            96744 non-null  float64
 8   completed              97109 non-null  int64  
 9   conference_game        97109 non-null  int64  
 10  home_team_id           97109 non-null  int64  
 11  home_team_full_name    97109 non-null  object 
 12  home_team_school_name  97109 non-null  object 
 13  home_team_mascot       96055 non-null  object 
 14  away_team_id           97109 non-null  int64  
 15  aw

In [142]:
empty_rows = df.isnull().any(axis=1).sum()
print(empty_rows)

94105


In [143]:
missing_values = df.isnull().sum()
print(missing_values)

id                           0
year                         0
week                         0
postseason                   0
id_home_team                 0
id_away_team                 0
points_home                365
points_away                365
completed                    0
conference_game              0
home_team_id                 0
home_team_full_name          0
home_team_school_name        0
home_team_mascot          1054
away_team_id                 0
away_team_full_name          0
away_team_school_name        0
away_team_mascot          3161
home_team_ranking        85746
away_team_ranking        88274
dtype: int64


In [144]:
df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97109 entries, 0 to 97108
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     97109 non-null  int64  
 1   year                   97109 non-null  int64  
 2   week                   97109 non-null  int64  
 3   postseason             97109 non-null  int64  
 4   id_home_team           97109 non-null  int64  
 5   id_away_team           97109 non-null  int64  
 6   points_home            96744 non-null  float64
 7   points_away            96744 non-null  float64
 8   completed              97109 non-null  int64  
 9   conference_game        97109 non-null  int64  
 10  home_team_id           97109 non-null  int64  
 11  home_team_full_name    97109 non-null  object 
 12  home_team_school_name  97109 non-null  object 
 13  home_team_mascot       96055 non-null  object 
 14  away_team_id           97109 non-null  int64  
 15  aw

In [145]:
duplicate_mask = df.duplicated(keep=False)
cleaned_df = df.drop_duplicates().copy()
print(df[duplicate_mask])

Empty DataFrame
Columns: [id, year, week, postseason, id_home_team, id_away_team, points_home, points_away, completed, conference_game, home_team_id, home_team_full_name, home_team_school_name, home_team_mascot, away_team_id, away_team_full_name, away_team_school_name, away_team_mascot, home_team_ranking, away_team_ranking]
Index: []


In [146]:
empty_rows = cleaned_df.isnull().any(axis=1).sum()
print(cleaned_df)

          id  year  week  postseason  id_home_team  id_away_team  points_home  \
0      21258  1936     6           0          2633           150         15.0   
1      21229  1936     6           0           108           159          7.0   
2      21245  1936     6           0           127       1000039          7.0   
3      21238  1936     6           0           344          2628          0.0   
4      21149  1936     5           0           221          2184          0.0   
...      ...   ...   ...         ...           ...           ...          ...   
97104  77708  2011    11           0          2506          2678         38.0   
97105  61829  1998     3           0           254            97         45.0   
97106  17583  1931     4           0           172       1000402         37.0   
97107  20026  1934    10           0          2751          2458          6.0   
97108  74557  2009     9           0           236          2717         24.0   

       points_away  complet

In [147]:
print(empty_rows)

94105


In [148]:
invalid_df = pd.DataFrame()
null_row_mask = df.isnull().any(axis=1)
rows_with_null = df[null_row_mask]
invalid_df = pd.concat([invalid_df, rows_with_null], ignore_index=True)
print(invalid_df)

          id  year  week  postseason  id_home_team  id_away_team  points_home  \
0      21229  1936     6           0           108           159          7.0   
1      21245  1936     6           0           127       1000039          7.0   
2      21238  1936     6           0           344          2628          0.0   
3      21140  1936     5           0          2102           218          7.0   
4      21089  1936     4           0           194           221          0.0   
...      ...   ...   ...         ...           ...           ...          ...   
94100  77708  2011    11           0          2506          2678         38.0   
94101  61829  1998     3           0           254            97         45.0   
94102  17583  1931     4           0           172       1000402         37.0   
94103  20026  1934    10           0          2751          2458          6.0   
94104  74557  2009     9           0           236          2717         24.0   

       points_away  complet

In [149]:
cleaned_df['home_team_mascot'].fillna('N/A', inplace=True)
cleaned_df['away_team_mascot'].fillna('N/A', inplace=True)
cleaned_df['home_team_ranking'].fillna(-1, inplace=True)
cleaned_df['away_team_ranking'].fillna(-1, inplace=True)
cleaned_df.dropna(inplace=True)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96744 entries, 0 to 97108
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     96744 non-null  int64  
 1   year                   96744 non-null  int64  
 2   week                   96744 non-null  int64  
 3   postseason             96744 non-null  int64  
 4   id_home_team           96744 non-null  int64  
 5   id_away_team           96744 non-null  int64  
 6   points_home            96744 non-null  float64
 7   points_away            96744 non-null  float64
 8   completed              96744 non-null  int64  
 9   conference_game        96744 non-null  int64  
 10  home_team_id           96744 non-null  int64  
 11  home_team_full_name    96744 non-null  object 
 12  home_team_school_name  96744 non-null  object 
 13  home_team_mascot       96744 non-null  object 
 14  away_team_id           96744 non-null  int64  
 15  away_te

In [ ]:
# cleaned_df.to_csv('data/cleaned_cfb_data.csv', index=False)
# cleaned_df.to_json('data/cleaned_cfb_data.json', orient='records')